In [1]:
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Parameters
batch_size = 16
img_height = 224
img_width = 224
data_dir = '/content/drive/MyDrive/postnatal'


Mounted at /content/drive


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG19, ResNet50, DenseNet121, MobileNetV2
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load and prepare dataset (as previously done)
batch_size = 16
img_height = 224
img_width = 224
data_dir = '/content/drive/MyDrive/postnatal'

dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    seed=123
)
class_names = dataset.class_names
print(f"Class names: {class_names}")

# Split dataset into train, val, test
train_size = 0.8
val_size = 0.1
test_size = 0.1

total_count = len(dataset)
train_count = int(total_count * train_size)
val_count = int(total_count * val_size)

train_ds = dataset.take(train_count)
val_ds = dataset.skip(train_count).take(val_count)
test_ds = dataset.skip(train_count + val_count)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

# Function to create a model
def create_model(base_model, input_shape, num_classes, freeze=True):
    base_model.trainable = not freeze
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create individual models
vgg19_base = VGG19(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
resnet50_base = ResNet50(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')


vgg19_model = create_model(vgg19_base, (img_height, img_width, 3), len(class_names))
resnet50_model = create_model(resnet50_base, (img_height, img_width, 3), len(class_names))

# Train individual models
epochs = 10
vgg19_history = vgg19_model.fit(train_ds, validation_data=val_ds, epochs=epochs)
resnet50_history = resnet50_model.fit(train_ds, validation_data=val_ds, epochs=epochs)


# Evaluate models
vgg19_acc = vgg19_model.evaluate(test_ds)[1]
resnet50_acc = resnet50_model.evaluate(test_ds)[1]

print(f"VGG19 Test Accuracy: {vgg19_acc:.2f}")
print(f"ResNet50 Test Accuracy: {resnet50_acc:.2f}")

# Combine predictions (Ensemble)
def ensemble_predictions(models, test_ds):
    all_preds = []
    all_true = []

    for images, labels in test_ds:
        all_true.extend(labels.numpy())
        model_preds = np.zeros((images.shape[0], len(class_names)))

        # Collect predictions from each model
        for model in models:
            preds = model.predict(images)
            model_preds += preds  # Sum the probabilities

        # Average the predictions (for soft-voting)
        avg_preds = model_preds / len(models)
        all_preds.extend(np.argmax(avg_preds, axis=1))

    return np.array(all_true), np.array(all_preds)

# Ensemble models
models = [vgg19_model, resnet50_model, ]
y_true, y_pred = ensemble_predictions(models, test_ds)

# Classification report
print("\nEnsemble Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

Found 2999 files belonging to 2 classes.
Class names: ['downSyndorme', 'healty']
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 615s 4s/step - accuracy: 0.6056 - loss: 1.7083 - val_accuracy: 0.7049 - val_loss: 0.5779
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 67s 447ms/step - accuracy: 0.7194 - loss: 0.5548 - val_accuracy: 0.6840 - val_loss: 0.5864
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 56s 375ms/step - accuracy: 0.7359 - loss: 0.5398 - val_accuracy: 0.6944 - val_loss: 0.5940
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 81s 371ms/step - accuracy: 0.7511 - loss: 0.4950 - val_accuracy: 0.6944 - val_loss: 0.6088
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 55s 366ms/step - accuracy: 0.7479 - loss: 0.4981 - val_accuracy: 0.7083 - val_loss: 0.5613
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 53s 355ms/step - accuracy: 0.7740 - loss: 0.4582 - val_accuracy: 0.7049 - val_loss: 0.6188
Epoch 7/10
150/150 ━━━━━━━━━━━━━━